<a href="https://colab.research.google.com/github/eflatlan/CNN_PID/blob/dev_floatmap/Dense_radius_notCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install h5py numpy

import os
import h5py
import numpy as np

from google.colab import drive
drive.mount('/content/drive')



In [ ]:
#!wget https://raw.githubusercontent.com/eflatlan/CNN_PID/dev_floatmap/helper_functions.py
#from helper_functions.py import print_points, plot_mapsm

In [ ]:

def calc_dist2mip(maps = None,  mip_positions = None, resolution = 10):
#        X_dist2mip = calc_dist2mip(maps = X_map, mip_positions = X_mip_position, resolution = resolution)

    length = maps.shape[0]
    distances_map_list = []



    #i = 0
    for i in range (length):

        map = np.array(maps[i, :,:])
        mip_pos = np.array(mip_positions[i, :]).copy()

        #print(f"filled_bins shape = {filled_bins.shape}")
        #print(f"map shape = {map.shape}")
        #print(f"mip_pos shape = {mip_pos.shape}")

        _mip_position = []


        # start vectorization
        # Assuming your map is a numpy array
        indices = np.where(map == 1)

        # Now indices[0] contains the y-indices and indices[1] contains the x-indices
        points = np.stack(indices, axis=-1)  # Shape is [num_points, 2]
        mip_pos = np.array([mip_pos[1], mip_pos[0]]) * resolution
        distances = np.linalg.norm(points - mip_pos, axis=-1)
        distances = distances[distances < 20*resolution]  # Filter out large distances
                                                          # later do this instead by imposing the masshypothesis
        # end vectorization

        #distances_map = []
        #for y in range(map.shape[0]):
        #    for x in range(map.shape[1]):
        #        if map[y, x] == 1:
        #            point = (x, y)
        #            distance = np.linalg.norm(np.array(point) - mip_pos*resolution)
        #            if distance < 80*resolution: # dont add if distance is unreasonably large
        #              distances_map.append(distance)
      
      
        distances_map_list.append(distances)
       # print(f"calc_dist2mip : distances : {distances}")
       # print(f"calc_dist2mip : mip_pos * resolution : {mip_pos}")
       # print(f"calc_dist2mip : points : {points}")

    # TODO: NB this should be removed, if i dont add it distances_map_list is one elem shorter than the other dataframes
      
    #distances_map_list.append(temp)


    print(f"maps shape = {maps.shape}")
    print(f"mip_position_array shape = {mip_positions.shape}")
    print(f"distances_map_list shape = {np.array(distances_map_list, dtype=object).shape}")

    return distances_map_list

In [ ]:
import math

class Constants:
    def __init__(self):
        self.defaultPhotonEnergy = 6.75
        self.CH4GapWidth = 8.0
        self.RadiatorWidth = 1.0
        self.QuartzWindowWidth = 0.5
        self.EmissionLength = self.RadiatorWidth / 2.0

class Masses:
    def __init__(self):
        self.mass_Pion = 0.1396
        self.mass_Kaon = 0.4937
        self.mass_Proton = 0.938


# Create constants and masses objects outside of CkovCalculator
constants = Constants()
masses = Masses()

class CkovCalculator:
    #def __init__(self):
        #self.constants = constants
        #self.masses = masses
    def GetFreonIndexOfRefraction(self, photonEnergy):
        x = photonEnergy
        k = 1.177 + (0.0172) * x
        return k

    def GetQuartzIndexOfRefraction(self, x):
        k = math.sqrt(1 + 46.411 / (113.763556 - x) + 228.71 / (328.51563 - x))
        return k

    def getRadiusFromCkov(self, ckovAngle):
        refIndexFreon = self.GetFreonIndexOfRefraction(constants.defaultPhotonEnergy)
        refIndexQuartz = self.GetQuartzIndexOfRefraction(constants.defaultPhotonEnergy)
        refIndexCH4 = 1.0

        sin_ckov = math.sin(ckovAngle)
        sin_qz = sin_ckov * (refIndexFreon / refIndexQuartz)
        sin_theta0 = sin_qz * (refIndexQuartz / refIndexCH4)

        R_ckov = sin_ckov * (constants.RadiatorWidth - constants.EmissionLength)
        R_qz = sin_qz * constants.QuartzWindowWidth
        R_0 = sin_theta0 * constants.CH4GapWidth

        R = R_ckov + R_qz + R_0
        return R

    def calcCkovFromMass(self, p, n):
        p_sq = p * p
        cos_ckov_denom = p * n

        radiuses = {}

        for particle, mass in masses.__dict__.items():
            # Skip non-mass attributes
            if not particle.startswith("mass"):
                continue

            mass_value = getattr(masses, particle)

            # sanity check
            if p_sq + mass_value * mass_value < 0:
                radiuses[particle] = 0.0
                continue

            cos_ckov = math.sqrt(p_sq + mass_value * mass_value) / cos_ckov_denom

            # sanity check
            if cos_ckov > 1 or cos_ckov < -1:
                radiuses[particle] = 0.0
                continue

            ckovAngle = math.acos(cos_ckov)
            radius = self.getRadiusFromCkov(ckovAngle)
            radiuses[particle] = radius
            #print(f"{particle} mass_value = {mass_value} Radius: {radius} ckov = {ckovAngle} momentum = {p} refindex = {n}")

        return radiuses


# Usage within a loop
for i in range(10):
    calculator = CkovCalculator()
    momentum = 1.5
    refractive_index = 1.289
    results = calculator.calcCkovFromMass(momentum, refractive_index)
    
    radiuses = results

    radiuses = []

    for particle, radius in results.items():
        #print(f"{particle} Radius: {radius}")
        radiuses.append(radius)

        

In [ ]:
#@title Default title text
from __future__ import print_function
import numpy as np
import matplotlib.pyplot as plt
from numpy.linalg import norm
import os
import h5py
import tensorflow as tf

# to check the impact of resolution in the 2d-map; 
# print the difference between the filledBins vector versus the map (map is restricted by resolution)
def print_points(filled_bins_array = None, map_array = None, mip_position_array = None, resolution = 10):

    length = map_array.shape[0]
    distances_bins_list = []
    distances_map_list = []

    print(f"filled_bins_array shape = {filled_bins_array.shape}")
    print(f"map_array shape = {map_array.shape}")
    print(f"mip_position_array shape = {mip_position_array.shape}")


    for i in range (1, length):

        filled_bins = np.array(filled_bins_array[i])
        map = np.array(map_array[i, :,:])
        mip_pos = np.array(mip_position_array[i, :])

        #print(f"filled_bins shape = {filled_bins.shape}")
        #print(f"map shape = {map.shape}")
        #print(f"mip_pos shape = {mip_pos.shape}")

        _mip_position = []
        #_mip_position.append(mip_position_array[])
        distances2 = []

        distances_bins = [norm(np.array(pos) - mip_pos) for pos in filled_bins]

        distances_map = []
        for y in range(map.shape[0]):
            for x in range(map.shape[1]):
                if map[y, x] == 1:
                    point = (x, y)
                    distance = np.linalg.norm(np.array(point) - mip_pos*resolution)
                    distances_map.append(distance)
        
        
        
        distances_bins_list.append(distances_bins)
        distances_map_list.append(distances_map)


    # Print the distances for each element in map_data_list
    print(f"Element {i+1} distances:")
    for j, (distances_bins, distances_map) in enumerate(zip(distances_bins_list, distances_map_list)):
        print(f"  Point {j+1}: Distance bins: {distances_bins}\n, Distance map: {distances_map}")
    print()




def plot_maps(filled_bins_array=None, map_array=None, mip_position_array=None, X_momentum=None, X_refractive_index=None, X_ckov=None, percentage_to_plot=5, resolution = 10):
  """
  Args : filled_bins_array : array that holds the vectors of filled pads 
         map_array : 2d  map with a determined resolution (the points in the filled_bins_array element, just restricted by the resolution)
         mip_position_array : array of the MIP {x, y} positions

         TODO : add mass_category and actual mass?
  """
  

  #percentage_to_plot = 0.05 / 10

  # Calculate the starting index of the samples to plot
  num_samples = map_array.shape[0]
  start_index = -num_samples

  # Create a subplot with the number of rows based on the number of samples
  fig, axes = plt.subplots(nrows=5, ncols=1, figsize=(8, 20))

  # Iterate over the samples and plot each map with information
  for i, ax in enumerate(axes):
      # Get the map and corresponding information
      map_data = map_array[start_index + i, :, :]
      #mass_category = particle_vector[start_index + i].mass_category
      ckov = X_ckov[start_index + i]
      mip_position = mip_position_array[start_index + i]
      momentum = X_momentum[start_index + i]
      refractive_index = X_refractive_index[start_index + i]

      # Plot the map
      ax.imshow(map_data, cmap='gray')



      #try :
      # Add a red dot at the MIP position
      ax.plot(mip_position[0]*resolution, mip_position[1]*resolution, 'ro')
      #Except exception as e : 
      #  print("caught non mip pos ")
      # Set the title with the information
      #ax.set_title(f"Mass: {mass_category}, CKOV: {ckov}, MIP Position: {mip_position}, Momentum: {momentum},  refractive_index: {refractive_index}")
      ax.set_title(f"CKOV: {ckov}, MIP Position: {mip_position}, Momentum: {momentum},  refractive_index: {refractive_index}")

      ax.axis('off')

  # Adjust the spacing between subplots
  plt.tight_layout()

  # Show the plot
  plt.show()

In [ ]:
import matplotlib.pyplot as plt
import random

def plot_random_element(X_train_map):
    index = random.randint(0, len(X_train_map) - 1)  # Pick a random index
    element = X_train_map[index, :, :, 0]  # Retrieve the element

    plt.figure(figsize=(8, 6))
    plt.imshow(element, cmap='viridis', origin='lower')
    plt.title(f"Random Element from X_train_map (Index {index})")
    plt.colorbar(label='Intensity')
    plt.xlabel('X Axis')
    plt.ylabel('Y Axis')
    plt.show()


    



In [ ]:
def plot5(X_test_map, particle_vector):

  # Plotting random maps with information

  # Select 5 random indices from the test data
  random_indices = np.random.choice(range(X_test_map.shape[0]), size=5, replace=False)

  # Create a subplot with 5 rows and 1 column
  fig, axes = plt.subplots(nrows=5, ncols=1, figsize=(8, 20))

  # Iterate over the random indices and plot each map with information
  for i, index in enumerate(random_indices):
      # Get the map and corresponding information
      map_data = X_test_map[index, :, :, 0]
      mass_category = particle_vector[index].mass_category
      ckov = particle_vector[index].ckov
      mip_position = particle_vector[index].mip_position
      momentum = particle_vector[index].momentum
      
      # Plot the map
      axes[i].imshow(map_data, cmap='gray')
      
      # Add a red dot at the MIP position
      axes[i].plot(mip_position[0], mip_position[1], 'ro')
      
      # Set the title with the information    a
      #x.set_title(f"Mass: {mass_category}, CKOV: {ckov}, MIP Position: {mip_position:.4f}, Momentum: {momentum:.4f}")
      mip_pos = f"{mip_position:.4f}"
      axes[i].set_title(f"Mass: {mass_category}, CKOV: {ckov}, MIP Position: {mip_pos}, Momentum: {momentum}")
      axes[i].axis('off')

  # Adjust the spacing between subplots
  plt.tight_layout()

  # Show the plot
  plt.show()

In [ ]:
def create_lr_scheduler(num_epochs = 10):

  start_lr = 0.1
  end_lr = 5e-6
  exp_decay = -np.log(end_lr/start_lr) / num_epochs # Calculate decay rate based on start and end learning rate

  lr_scheduler = tf.keras.callbacks.LearningRateScheduler(lambda epoch: start_lr * np.exp(-exp_decay * epoch)) 
  return lr_scheduler





def plot_lr(num_epochs = 10, history = None):
  div = num_epochs/4
  lrs = 1e-4 * (10 ** (np.arange(num_epochs)/div))
  plt.figure(figsize=(10, 7))
  plt.semilogx(lrs, history.history["loss"]) # we want the x-axis (learning rate) to be log scale
  plt.xlabel("Learning Rate")
  plt.ylabel("Loss")


  plt.title("Learning rate vs. loss");

In [ ]:
#def plot_worst_(model, y_test, X_test_map, X_test_momentum, X_test_refractive_index, X_test_ckov, X_test_mip_position, y_pred):
def plot_maps(filled_bins_array=None, map_array=None, mip_position_array=None, X_momentum=None, X_refractive_index=None, X_ckov=None, percentage_to_plot=5, resolution = 10):
  #  print("Shape of y_pred: ", y_pred.shape)
  # 1. Predict labels on validation data
  #plot_worst(model, y_test, X_test["X_test_map"], X_test["X_test_momentum"], X_test["X_test_refractive_index"], X_test["X_test_ckov"], X_test["X_test_mip_position"], y_pred_test)

  # 2. Calculate the difference between predicted and actual labels
  losses = tf.keras.losses.categorical_crossentropy(y_test, y_pred).numpy()

  # Sort the indices of the losses from highest to lowest
  sorted_indices = np.argsort(losses)[::-1]

  # Get the indices of the worst performing 10%
  worst_10_percent_indices = sorted_indices[:int(0.1*len(sorted_indices))]

  # Create figure and axes
  num_plots = len(worst_10_percent_indices)
  #fig, axes = plt.subplots(num_plots, 1, figsize=(8, 20))
  fig, axes = plt.subplots(num_plots,figsize=(8, 20))

  # Define mass categories
  mass_categories = ["pion", "kaon", "proton"]

  # 3. Create plots for these cases, including their feature information and predicted vs actual labels
  for i, index in enumerate(worst_10_percent_indices):
      # Get the map and corresponding information
      map_data = X_test_map[index, :, :]
      actual_mass_category = mass_categories[np.argmax(y_test[index])]

      print(f"y_test[index] = {y_test[index]}")

      predicted_mass_category = mass_categories[np.argmax(y_pred[index])]
      ckov = X_test_ckov[index]
      mip_position = X_test_mip_position[index]
      momentum = X_test_momentum[index]
      refractive_index = X_test_refractive_index[index]
      
      mass_actual = momentum * np.sqrt(refractive_index**2 * np.cos(ckov)*np.cos(ckov) - 1)
      
      # Check if the value is NaN (invalid Cherenkov angle)
      if np.isnan(mass_actual):
          mass_actual = "Invalid"

      # Plot the map
      axes[i].imshow(map_data, cmap='gray')

      # Add a red dot at the MIP position
      axes[i].plot(mip_position[0]*resolution, mip_position[1]*resolution, 'ro')

      # Set the title with the information
      axes[i].set_title(f"Actual Mass")#: {actual_mass_category}, Predicted Mass: {predicted_mass_category},\nMass: {mass_actual}, Mass_prob = {y_pred[index]} \nCKOV: {ckov}, MIP Position: {mip_position}, \nMomentum: {momentum}, Refractive Index: {refractive_index}")
      #
      axes[i].set_title(f"Actual Mass: {actual_mass_category}, Predicted Mass: {predicted_mass_category},\nMass: {mass_actual}, Mass_prob = {y_pred[index]} \nCKOV: {ckov}, MIP Position: {mip_position}, \nMomentum: {momentum}, Refractive Index: {refractive_index}")

      #axes[i].set_title(f"Actual Mass: {actual_mass_category}, Predicted Mass: {predicted_mass_category}, Mass: {mass_actual}\nCKOV: {ckov}, MIP Position: {mip_position}, Momentum: {momentum}, Refractive Index: {refractive_index}")
      axes[i].axis('off')

      print("\n")
      print(f"  Actual Mass: {actual_mass_category}, Predicted Mass: {predicted_mass_category},\n Mass: {mass_actual}, Mass_prob = {y_pred[index]} \n CKOV: {ckov}, MIP Position: {mip_position}, \n  Momentum: {momentum}, Refractive Index: {refractive_index}")
  # Adjust the spacing between subplots
  plt.tight_layout()

  # Show the plot
  plt.show()


In [ ]:
def create_circular_mask(center, size, mean_radius, std):
    H, W = size
    Y, X = np.ogrid[:H, :W]
    dist_from_center = np.sqrt((X - center[0])**2 + (Y-center[1])**2)

    mask = (dist_from_center >= mean_radius - 2*std) & (dist_from_center <= mean_radius + 2*std)
    return mask


def extract_segment_around_mip(mip_positions = None, window_sizes = None, maps = None, std = 7):
    """
    Args : mip_positions : array of the MIP-positions, (num_samples, {x, y})
           window_sizes : radius of the segments (num_samples, 3) 3 = number of particle classes
           maps : the photon hit-maps (num_samples, 144*resolution, 160*resolution)
           std : standard deviatons to be applied to the ring-radius 
           
    Returns : the extracted regions for each of the different radiuses (m_pion, m_kaon, m_proton), masked with the map
    """
    
    windows = []

    for mip_position, window_size, map in zip(mip_positions, window_sizes, maps):
        radius = np.mean(window_size)
        #std = np.std(window_size)

        mask = create_circular_mask(mip_position, map.shape, radius, std)

        window = np.where(mask, map, 0)

        windows.append(window)

    return np.array(windows)


In [ ]:
#resolution = 4

print_vals = False
from numpy.linalg import norm
from tensorflow.keras.backend import expand_dims
from tensorflow.keras.preprocessing.sequence import pad_sequences

from scipy.signal import find_peaks

from __future__ import print_function
import os
import h5py
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Activation, Input, Conv2D, Lambda, Flatten, Dense, concatenate, BatchNormalization, MaxPooling2D, Dropout, LeakyReLU, Masking
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelBinarizer
import matplotlib.pyplot as plt
from tensorflow.keras import regularizers

class Constants:
    PION_MASS = 0.1396
    KAON_MASS = 0.4937
    PROTON_MASS = 0.938

np.set_printoptions(precision=4)

@staticmethod
def calculate_mass(momentum, refractiveIndex, ckov):
    """ args : momentum, refractiveIndex, ckov
        returns : mass
    """
    mass = momentum * np.sqrt((refractiveIndex * np.cos(ckov))**2 - 1)
    return mass


class ParticleDataUtils:
  
    class ParticleInfo:
        def __init__(self, momentum, mass, refractiveIndex, ckov, filledBins, mip_position, radiuses):

            self.momentum = momentum
            self.mass = mass
            self.refractiveIndex = refractiveIndex
            self.ckov = ckov
            self.filledBins = filledBins
            self.mip_position = mip_position
            self.mass_category = self.infer_mass_category_from_ckov(momentum, refractiveIndex, ckov)  # Infer mass category based on mass
            self.distances_to_mip = self.calculate_distances_to_mip()  # Calculate distances
            self.radiuses = radiuses

        @staticmethod
        def infer_mass_category_from_ckov(momentum, refractiveIndex, ckov):
            mass = momentum * np.sqrt((refractiveIndex * np.cos(ckov))**2 - 1)
            
            mass_category = "unknown"
            if abs(mass - Constants.PION_MASS) < 1e-4:
                mass_category = "pion"
            elif abs(mass - Constants.KAON_MASS) < 1e-4:
                mass_category = "kaon"
            elif abs(mass - Constants.PROTON_MASS) < 1e-4:
                mass_category = "proton"
            if print_vals:
              print(f"\ninfer_mass_category_from_ckov :  momentum = {momentum}|  mass_calc = {mass} |  mass_category={mass_category} | refractiveIndex = {refractiveIndex} | ckov = {ckov}")
            return mass_category

        @staticmethod
        def infer_mass_category(mass):
            if abs(mass - Constants.PION_MASS) < 1e-6:
                return "pion"
            elif abs(mass - Constants.KAON_MASS) < 1e-6:
                return "kaon"
            elif abs(mass - Constants.PROTON_MASS) < 1e-6:
                return "proton"
            else:
                return "unknown"

        def __str__(self):
            if print_vals:
              return (f"ParticleInfo(momentum={self.momentum} | mass={self.mass} |  mass_category={self.mass_category} | "
                      f"refractiveIndex={self.refractiveIndex} | ckov={self.ckov} | num_filled_bins={len(self.filledBins)}, "
                      f"mip_position={self.mip_position})")

        def calculate_distances_to_mip(self):
            """Calculate Euclidean distances from all filled bins to MIP position"""
            filledBins_np = np.array(self.filledBins)
            mip_position_np = np.array(self.mip_position)

            distances = np.linalg.norm(filledBins_np - mip_position_np, axis=1)
            return distances


    def __init__(self, filename = "default_filename.h5", percentage_to_read = 100):
        self.filename = filename
        self.percentage_to_read = percentage_to_read
        self.particle_vector = self.load_data(filename)
        self.particle_info = self.process_data(self.particle_vector, self.percentage_to_read)
        self.num_particles = len(self.particle_info)
        self.momentum_scaler, self.momentum_stats = self.create_scaler("momentum")
        self.refractive_index_scaler, self.refractive_index_stats = self.create_scaler("refractiveIndex")
        self.ckov_scaler, self.ckov_stats = self.create_scaler("ckov")
        self.distances_scaler, self.distances_stats = self.create_scaler("distances")


    def load_data(self, filename):
        drive_path = '/content/drive/MyDrive/Colab Notebooks/CERN_ML/CNN_PID/'  # Update the path to your Google Drive folder
        file_path = os.path.join(drive_path, filename)
        particle_vector = []
        with h5py.File(file_path, 'r') as file:
            #file.visititems(print_hdf5_items)
            for i, group_name in enumerate(file):
                group = file[group_name]

                # Read scalar values
                momentum = group.attrs['Momentum']
                mass = group.attrs['Mass']
                refractiveIndex = group.attrs['RefractiveIndex']
                ckov = group.attrs['Ckov']
                mip_position = group['MipPos']
                mip_position = mip_position[...]  # Retrieve the data as a numpy array

                mip_position = mip_position.tolist()  # Convert the numpy array to a list

                # Read filledBins
                filled_bins_dataset = group['FilledBins']
                filled_bins_data = filled_bins_dataset[...]  # Retrieve the data as a numpy array

                filled_bins = filled_bins_data.tolist()  # Convert the numpy array to a list

                # get window : 
                calculator = CkovCalculator()
                results = calculator.calcCkovFromMass(momentum, refractive_index)
                radiuses = []
                for particle, radius in results.items():
                    #print(f"{particle} Radius: {radius}")
                    radiuses.append(radius)

                # the biggest radius should then extract a window around the MIP 

                # Find the particle with the biggest radius
                #max_radius_particle = max(radiuses, key=radiuses.get)
                #max_radius = radiuses[max_radius_particle]
                max_radius = radiuses[2]

                # Extract a window around the MIP using the biggest radius
                #window_size = int(max_radius * 2)  # Adjust the size as needed      
                # NB!!! add resolution here? Or multiply elswhere..

                particle_info = ParticleDataUtils.ParticleInfo(
                    momentum, mass, refractiveIndex, ckov, filledBins=filled_bins, mip_position=mip_position, radiuses = radiuses)
                if print_vals == True:
                  print(particle_info)  # This will use the __str__() method of ParticleInfo

                particle_vector.append(particle_info)

        return particle_vector

    def process_data(self, particle_vector, percentage):

        # Calculate the number of particles based on the percentage
        num_particles = int(len(particle_vector) * (percentage / 100.0))

        # Slice the particle_vector to the desired percentage
        particle_vector = particle_vector[:num_particles]
        return particle_vector


    def create_scaler(self, feature):
        if feature == "momentum":
            values = np.array([info.momentum for info in self.particle_info]).reshape(-1, 1)
        elif feature == "refractiveIndex":
            values = np.array([info.refractiveIndex for info in self.particle_info]).reshape(-1, 1)
        elif feature == "ckov":
            values = np.array([info.ckov for info in self.particle_info]).reshape(-1, 1)
        elif feature == "distances":
            distances = []
            for info in self.particle_info:
                distances.extend(info.distances_to_mip)
            values = np.array(distances).reshape(-1, 1)
        else:
            raise ValueError(f"Invalid feature: {feature}")

        scaler = StandardScaler()
        scaled_values = scaler.fit_transform(values)
        stats = {
            "mean": scaler.mean_[0],
            "std": scaler.scale_[0]
        }
        return scaler, stats



# create a map, the resolution is the "inverse" 
def create_map(filledBins=None, resolution=4):
    map_shape = (int(144 * resolution), int(160 * resolution))
    map_data = np.zeros(map_shape, dtype=np.int32)
   
    if filledBins is not None:
        filledBins_np = np.array(filledBins)
        indices = (filledBins_np * resolution).astype(int)
        map_data[indices[:, 1], indices[:, 0]] = 1

    return map_data

class MassClassifier:
    def __init__(self, percentage_to_read = 10, resolution = 4):
        self.model = None
        self.utils = None
        self.percentage_to_read = percentage_to_read
        self.resolution = resolution


    def load_data(self, filename):
        self.utils = ParticleDataUtils(filename, percentage_to_read = self.percentage_to_read) # specify percentage of particles to read..
        print(f"Number of particles: {self.utils.num_particles}")

    def preprocess_data(self):
        particle_info = self.utils.particle_info


        # Prepare the inputs
        
        # create a map with resolution to be chosen, iterate over teh filledBins vector
        X_map = np.array([create_map(filledBins = info.filledBins, resolution = self.resolution) for info in particle_info])
        filled_bins_array = np.array([info.filledBins for info in particle_info], dtype=object)



        X_momentum = np.array([info.momentum for info in particle_info])#.reshape(-1, 32, 32, 1)
        X_refractive_index = np.array([info.refractiveIndex for info in particle_info])#.reshape(-1, 32, 32, 1)
        X_ckov = np.array([info.ckov for info in particle_info])#.reshape(-1, 32, 32, 1)
        X_mip_position = np.array([info.mip_position for info in particle_info])

        # calculate radiuses (radius of f(m_i) where m_i = [m_pion, m_kaon, m_proton],  and n, p, + later {xRad, yRad, theta, phi} fixed for a given track) 
        # for mass-hypothesis

        #alc_dist2mip(maps = None,  mip_positions = None, resolution = 10):
        
        try:
          plot_maps(filled_bins_array=filled_bins_array, map_array=X_map, mip_position_array=X_mip_position, X_momentum=X_momentum, X_refractive_index=X_refractive_index, X_ckov=X_ckov, percentage_to_plot=5, resolution = self.resolution)
        except Exception as e:
          print(f"plot_maps failed due to error : {e}")
        
        # change shape to be num_samples, pad_sequences, 3:



        X_dist2mip = calc_dist2mip(maps = X_map, mip_positions = X_mip_position, resolution = self.resolution)

        print(f" X_dist2mip shape befoer pad = {np.array(X_dist2mip).shape}")


        X_dist2mip = pad_sequences(X_dist2mip, padding='post')
        print(f" X_dist2mip shape after pad = {np.array(X_dist2mip).shape}")

        # extracting the window around the MIP:

        #  extract_segment_around_mip(mip_positions = None, radiuses = None, maps = None, std = 7):
        #X_windows = extract_segment_around_mip(mip_positions = X_mip_position, radiuses = X_radiuses, maps = X_map, std = 7)
        
        # this adds wrong? denne dimensionen er en for stor allerede?
        #X_windows = np.expand_dims(X_windows, -1) 
        #print(f"X_windows shape = {np.array(X_windows).shape}")
        # X_windows will have various size (X_window_sizes + pad) x (X_window_sizes + pad) 


        # check the impact of resolution:
        #print_points(filled_bins_array = filled_bins_array, map_array = X_map, mip_position_array= X_mip_position, resolution = resolution)

        # plot the points and the MIP
        #plot_maps(filled_bins_array =filled_bins_array, map_array = X_map, mip_position_array= X_mip_position, X_momentum = X_momentum, X_refractive_index= X_refractive_index, X_ckov = X_ckov, percentage_to_plot=5, resolution = resolution)


        # Normalize the inputs NB commented out scaling !!!
        #X_momentum = self.utils.momentum_scaler.transform(X_momentum.reshape(-1, 1))#.reshape(-1, 32, 32, 1)
        #X_refractive_index = self.utils.refractive_index_scaler.transform(X_refractive_index.reshape(-1, 1))#.reshape(-1, 32, 32, 1)
        #X_ckov = self.utils.ckov_scaler.transform(X_ckov.reshape(-1, 1))#.reshape(-1, 32, 32, 1)

        #X_mip_position[0,:] = self.utils.distances_scaler.transform(X_mip_position[0,:])
        #X_mip_position[1,:] = self.utils.distances_scaler.transform(X_mip_position[1,:])



        # Prepare the outputs
        y = np.array([info.mass_category for info in particle_info])

        # Convert the outputs to one-hot encoded vectors
        lb = LabelBinarizer()
        y = lb.fit_transform(y)

        # Split the data into train and test sets

        X_train_dist2mip, X_test_dist2mip, X_train_map, X_test_map, X_train_momentum, X_test_momentum, X_train_refractive_index, X_test_refractive_index, \
            X_train_ckov, X_test_ckov, X_train_mip_position, X_test_mip_position, y_train, y_test = \
            train_test_split(X_dist2mip, X_map, X_momentum, X_refractive_index, X_ckov, X_mip_position, y, test_size=0.2, random_state=42)

        X_train = {"X_train_map": X_train_map, "X_train_momentum": X_train_momentum, "X_train_refractive_index": X_train_refractive_index,
                   "X_train_ckov": X_train_ckov, "X_train_mip_position": X_train_mip_position, "X_train_dist2mip": X_train_dist2mip}

        X_test = {"X_test_map": X_test_map, "X_test_momentum": X_test_momentum, "X_test_refractive_index": X_test_refractive_index,
                  "X_test_ckov": X_test_ckov, "X_test_mip_position": X_test_mip_position, "X_test_dist2mip": X_test_dist2mip}

        return (X_train, X_test, y_train, y_test)

    def build_model(self, input_sequence_length = None):

        #map_shape = (None, None, 1)  # Variable shape for the map input

        momentum_shape = (1,)
        refractive_index_shape = (1,)
        mip_position_shape = (2,)

        # Grid search parameters
        # Grid search parameters
        filter_sizes = [5]  # Filter sizes to test
        num_filters = [32]#[16, 32]  # Number of filters to test
        strides = [(2, 2)]#[(1, 1), (2, 2)]  # Strides to test
        pool_sizes = [(2, 2)]  # Max pooling sizes to test
        fc1_units = [64]#, 128]  # Number of units in fc1 to test
        fc2_units = [32, 64]#, 32]  # Number of units in fc2 to test

        dropouts = [0.2, 0.3, 0.4]
        best_accuracy = 0
        best_model = None

        for dropout in dropouts:
            for num_filter in num_filters:
                for stride in strides:
                    for pool_size in pool_sizes:
                        for fc1_unit in fc1_units:
                            for fc2_unit in fc2_units:
                              # Create a new model configuration

                              # X_dist2mip



                              #dist2mip_input = Input(shape=dist2mip_shape, name="dist2mip_input")
                              momentum_input = Input(shape=momentum_shape, name="momentum_input")
                              refractive_index_input = Input(shape=refractive_index_shape, name="refractive_index_input")
                              mip_position_input = Input(shape=mip_position_shape, name="mip_position_input")



                              # Distance to MIP:
                              dist2mip_input = Input(shape=(input_sequence_length,), name="dist2mip_input")

                              # to ignore the padded values:
                              dist2mip_masked = Masking(mask_value=0.)(dist2mip_input)

                              dense_dist2mip_1 = Dense(128, name="dense_dist2mip_1")(dist2mip_masked)
                              bn_dist2mip_1 = BatchNormalization(name="bn_dist2mip_1")(dense_dist2mip_1)
                              relu_dist2mip_1 = Activation('relu', name="relu_dist2mip_1")(bn_dist2mip_1)
                              dropout_dist2mip_1 = Dropout(dropout, name="dropout_dist2mip_1")(relu_dist2mip_1)

                              dense_dist2mip_2 = Dense(64, name="dense_dist2mip_2")(dropout_dist2mip_1)
                              bn_dist2mip_2 = BatchNormalization(name="bn_dist2mip_2")(dense_dist2mip_2)
                              relu_dist2mip_2 = Activation('relu', name="relu_dist2mip_2")(bn_dist2mip_2)
                              dropout_dist2mip_2 = Dropout(0.2, name="dropout_dist2mip_2")(relu_dist2mip_2)

                              dense_dist2mip_3 = Dense(32, name="dense_dist2mip_3")(dropout_dist2mip_2)
                              bn_dist2mip_3 = BatchNormalization(name="bn_dist2mip_3")(dense_dist2mip_3)
                              relu_dist2mip_3 = Activation('relu', name="relu_dist2mip_3")(bn_dist2mip_3)
                              dropout_dist2mip_3 = Dropout(0.2, name="dropout_dist2mip_3")(relu_dist2mip_3)

                              dense_dist2mip_4 = Dense(16, name="dense_dist2mip_4")(dropout_dist2mip_3)
                              bn_dist2mip_4 = BatchNormalization(name="bn_dist2mip_4")(dense_dist2mip_4)
                              relu_dist2mip_4 = Activation('relu', name="relu_dist2mip_4")(bn_dist2mip_4)
                              dropout_dist2mip_4 = Dropout(0.2, name="dropout_dist2mip_4")(relu_dist2mip_4)


                              # NB! now skipping the BatchNormalization
                              # Refractive index Input
                              # Refractive index Input
                              dense_ref_index = Dense(fc1_unit, name="dense_ref_index")(refractive_index_input)
                              bn_ref_index = BatchNormalization(name="bn_ref_index")(dense_ref_index)
                              relu_ref_index = Activation('relu', name="relu_ref_index")(bn_ref_index)
                              dropout_ref_index = Dropout(dropout, name="dropout_ref_index")(relu_ref_index)

                              dense_ref_index_2 = Dense(fc1_unit/2, name="dense_ref_index_2")(dropout_ref_index)
                              bn_ref_index_2 = BatchNormalization(name="bn_ref_index_2")(dense_ref_index_2)
                              relu_ref_index_2 = Activation('relu', name="relu_ref_index_2")(bn_ref_index_2)
                              dropout_ref_index_2 = Dropout(dropout, name="dropout_ref_index_2")(relu_ref_index_2)


                              # Momentum Input
                              dense_momentum = Dense(fc1_unit, name="dense_momentum")(momentum_input)
                              bn_momentum = BatchNormalization(name="bn_momentum")(dense_momentum)
                              relu_momentum = Activation('relu', name="relu_momentum")(bn_momentum)
                              dropout_momentum = Dropout(dropout, name="dropout_momentum")(relu_momentum)

                              dense_momentum_2 = Dense(fc1_unit/2, name="dense_momentum_2")(dropout_momentum)
                              bn_momentum_2 = BatchNormalization(name="bn_momentum_2")(dense_momentum_2)
                              relu_momentum_2 = Activation('relu', name="relu_momentum_2")(bn_momentum_2)
                              dropout_momentum_2 = Dropout(dropout, name="dropout_momentum_2")(relu_momentum_2)


                              # MIP Position Input
                              dense_mip_pos = Dense(fc1_unit, name="dense_mip_pos")(mip_position_input)
                              bn_mip_pos = BatchNormalization(name="bn_mip_pos")(dense_mip_pos)
                              relu_mip_pos = Activation('relu', name="relu_mip_pos")(bn_mip_pos)
                              dropout_mip_pos = Dropout(dropout, name="dropout_mip_pos")(relu_mip_pos)

                              dense_mip_pos_2 = Dense(fc1_unit/2, name="dense_mip_pos_2")(dropout_mip_pos)
                              bn_mip_pos_2 = BatchNormalization(name="bn_mip_pos_2")(dense_mip_pos_2)
                              relu_mip_pos_2 = Activation('relu', name="relu_mip_pos_2")(bn_mip_pos_2)
                              dropout_mip_pos_2 = Dropout(dropout, name="dropout_mip_pos_2")(relu_mip_pos_2)

                              
                              epochs = 300
                              lr = create_lr_scheduler(num_epochs = epochs)

                              # Concatenation
                              concat = concatenate([dropout_dist2mip_3, dropout_ref_index_2, dropout_momentum_2, dropout_mip_pos_2])


                              dense_concat = Dense(16, name="dense_concat")(concat)
                              bn_concat = BatchNormalization(name="bn_concat")(dense_concat)
                              relu_concat = Activation('relu', name="relu_concat")(dense_mip_pos)
                              dropout_concat = Dropout(dropout, name="dropout_concat")(relu_concat)


                              output = Dense(3, activation='softmax')(concat) # NB LEGG MERKE TIL AT DENNE ER ENDRET FRA FC2

                              model = Model(inputs=[dist2mip_input, momentum_input, refractive_index_input, mip_position_input],
                                            outputs=output)

                              #model = Model(inputs=[window_input], outputs=[output])
                              model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005), # NB changed from 0.0002
                                            loss="categorical_crossentropy", metrics=["accuracy"])

                              X_train, X_test, y_train, y_test = self.preprocess_data()

                              # Train the model
                              history = model.fit(
                                  x=[X_train["X_train_dist2mip"], X_train["X_train_momentum"], X_train["X_train_refractive_index"],
                                      X_train["X_train_mip_position"]], # x=[X_train_dist2mip, X_train_momentum, X_train_refractive_index, X_train_ckov, X_train_mip_position]
                                  y=y_train,
                                  validation_data=(
                                      [X_test["X_test_dist2mip"], X_test["X_test_momentum"], X_test["X_test_refractive_index"],
                                        X_test["X_test_mip_position"]],
                                      y_test
                                  ),
                                  batch_size=32,
                                  epochs=epochs,
                                  verbose=1,
                                  callbacks = [lr]
                              )

                              # plotting the worst cases?:
                              #y_pred_test = model.predict([X_test["X_test_map"], X_test["X_test_momentum"], X_test["X_test_refractive_index"], X_test["X_test_mip_position"]])
                              #plot_worst(model, y_test, X_test_map, X_test_momentum, X_test_refractive_index, X_test_ckov, X_test_mip_position, y_pred):
                              print("Shape of y_test: ", y_test.shape)
                              print("Shape of X_test_map: ", X_test["X_test_map"].shape)
                             # print("Shape of X_test_windows: ", X_test["X_test_windows"].shape)
                              print("Shape of X_test_momentum: ", X_test["X_test_momentum"].shape)
                              print("Shape of X_test_refractive_index: ", X_test["X_test_refractive_index"].shape)
                              print("Shape of X_test_ckov: ", X_test["X_test_ckov"].shape)
                              print("Shape of X_test_mip_position: ", X_test["X_test_mip_position"].shape)
                              #print("Shape of y_pred_test: ", y_pred_test.shape)
                              try: # NB! commented out this line
                                #plot_worst_(model, y_test, X_test["X_test_map"], X_test["X_test_momentum"], X_test["X_test_refractive_index"], X_test["X_test_ckov"], X_test["X_test_mip_position"], y_pred_test)
                                plot_worst_(model, y_test, X_test["X_test_map"], X_test["X_test_momentum"], X_test["X_test_refractive_index"], X_test["X_test_ckov"], X_test["X_test_mip_position"], self.resolution)
                              except Exception as e:
                                print(f"skip plot_worst_ due to error: {e}")
                              self.plot_training_history(history)

                              # Evaluate the model
                              _, accuracy = model.evaluate(
                                  x=[X_test["X_test_dist2mip"], X_test["X_test_momentum"], X_test["X_test_refractive_index"],
                                        X_test["X_test_mip_position"]],
                                  y=y_test,
                                  verbose=1
                              )

                              print(f"Model Accuracy: {accuracy:.4f} (fc1 Size: {fc1_unit}, Num fc2: {fc2_unit}, dropout = {dropout}")


                              # Check if the current model configuration is better
                              if accuracy > best_accuracy:
                                  best_accuracy = accuracy
                                  best_model = model

        # Set the best model as the final model
        self.model = best_model


    def train_model(self, X_train, X_test, y_train, y_test):
        # Compile the model


        X_train_map = X_train["X_train_map"]
        X_train_dist2mip = X_train["X_train_dist2mip"]
        X_train_momentum = X_train["X_train_momentum"]
        X_train_refractive_index = X_train["X_train_refractive_index"]
        X_train_ckov = X_train["X_train_ckov"]
        X_train_mip_position = X_train["X_train_mip_position"]

        X_test_map = X_test["X_test_map"]
        X_test_dist2mip = X_test["X_test_dist2mip"]


        # Define the bin edges for each histogram.
        bins1 = np.arange(0, self.resolution*10.1, 0.01)
        bins2 = np.arange(4, self.resolution*8.1, 0.01)
        bins3 = np.arange(0, self.resolution*2.1, 0.01)

        # Create a figure with 3 subplots (one row, three columns).
        fig, axs = plt.subplots(1, 3, figsize=(15, 5))

        
        num_zeros = np.sum(X_train_dist2mip == 0)


        mask = (X_test_dist2mip >= self.resolution*1) & (X_test_dist2mip <= self.resolution*8)

        # Index your array with the mask
        selected_values = X_test_dist2mip[mask]
        num_elements = int(0.1 * len(selected_values))

        # Print the selected values
        first_10_percent = selected_values[:num_elements]

        # Print these elements
        print(selected_values)

        print(f'There are {num_zeros} zeros in X_train_dist2mip.')
        print(f"X_train_dist2mip shape = {np.array(X_train_dist2mip).shape}")

        # Plot the first histogram.
        axs[0].hist(X_test_dist2mip, bins=bins1, edgecolor='black')
        axs[0].set_title('Histogram 1 of X_test_dist2mip')
        axs[0].set_xlabel('Values')
        axs[0].set_ylabel('Frequency')

        # Plot the second histogram.
        axs[1].hist(X_test_dist2mip, bins=bins2, edgecolor='black')
        axs[1].set_title('Histogram 2 of X_test_dist2mip')
        axs[1].set_xlabel('Values')
        axs[1].set_ylabel('Frequency')
 
        # Plot the third histogram.
        axs[2].hist(X_test_dist2mip, bins=np.arange(0, .01, 0.001), edgecolor='black')
        axs[2].set_title('Histogram 3 of X_test_dist2mip')
        axs[2].set_xlabel('Values')
        axs[2].set_ylabel('Frequency')

        # Display the plot.
        plt.tight_layout()
        plt.show()


        # # Define the bin edges for each histogram.
        # bins11 = np.arange(self.resolution*0.5, self.resolution*10.1, 0.1)
        # bins22 = np.arange(self.resolution*0.5, self.resolution*2.1, 0.1)
        # bins33 = np.arange(self.resolution*0.5, self.resolution*5.1, 0.1)

        # # Create histograms without plotting, to identify peaks.
        # hist1, edges1 = np.histogram(X_test_dist2mip, bins=bins11)
        # hist2, edges2 = np.histogram(X_test_dist2mip, bins=bins22)
        # hist3, edges3 = np.histogram(X_test_dist2mip, bins=bins33)

        # # Find peaks (you might want to adjust the parameters).
        # peaks1, _ = find_peaks(hist1, height=10) # adjust the height as per your requirement
        # peaks2, _ = find_peaks(hist2, height=10)
        # peaks3, _ = find_peaks(hist3, height=10)

        # # Create a figure with 3 subplots (one row, three columns).
        # fig, axs = plt.subplots(1, 3, figsize=(15, 5))

        # # Plot the first histogram with peaks.
        # axs[0].hist(X_test_dist2mip, bins=bins11, edgecolor='black')
        # axs[0].plot(edges1[peaks1], hist1[peaks1], "ro") # Peaks marked in red
        # axs[0].set_title('Histogram 1 with Peaks')
        # axs[0].set_xlabel('Values')
        # axs[0].set_ylabel('Frequency')

        # # Plot the second histogram with peaks.
        # axs[1].hist(X_test_dist2mip, bins=bins22, edgecolor='black')
        # axs[1].plot(edges2[peaks2], hist2[peaks2], "ro") # Peaks marked in red
        # axs[1].set_title('Histogram 2 with Peaks')
        # axs[1].set_xlabel('Values')
        # axs[1].set_ylabel('Frequency')

        # # Plot the third histogram with peaks.
        # axs[2].hist(X_test_dist2mip, bins=bins33, edgecolor='black')
        # axs[2].plot(edges3[peaks3], hist3[peaks3], "ro") # Peaks marked in red
        # axs[2].set_title('Histogram 3 with Peaks')
        # axs[2].set_xlabel('Values')
        # axs[2].set_ylabel('Frequency')

        # # Display the plot.
        # plt.tight_layout()
        # plt.show()

        # for i in range(5):
        #   fig, axs = plt.subplots(1, 3, figsize=(15, 5))
        #   temp = X_test_dist2mip[:, i]
        #   max_value = np.max(temp)
        #   print(f"max_value = {max_value}")
        #   print(f"temp shape = {np.array(temp).shape}")
          
        #   # Plot the first histogram.
        #   axs[0].hist(temp, bins=bins1, edgecolor='black')
        #   axs[0].set_title(f'{i} X_test_dist2mip')
        #   axs[0].set_xlabel('Values')
        #   axs[0].set_ylabel('Frequency')

        #   # Plot the second histogram.
        #   axs[1].hist(temp, bins=bins2, edgecolor='black')
        #   axs[1].set_title(f'{i} X_test_dist2mip')
        #   axs[1].set_xlabel('Values')
        #   axs[1].set_ylabel('Frequency')

        #   # Plot the third histogram.
        #   axs[2].hist(temp, bins=bins3, edgecolor='black')
        #   axs[2].set_title(f'{i} X_test_dist2mip')
        #   axs[2].set_xlabel('Values')
        #   axs[2].set_ylabel('Frequency')

        #   # Display the plot.
        #   plt.tight_layout()
        #   plt.show()

        # for i in range(5):
        #   fig, axs = plt.subplots(1, 3, figsize=(15, 5))
        #   temp = X_test_dist2mip[i, :]
        #   max_value = np.max(temp)
        #   print(f"max_value = {max_value}")
        #   print(f"temp shape = {np.array(temp).shape}")

        #   # Plot the first histogram.
        #   axs[0].hist(temp, bins=bins1, edgecolor='black')
        #   axs[0].set_title(f'{i} a X_test_dist2mip')
        #   axs[0].set_xlabel('a Values')
        #   axs[0].set_ylabel('Frequency')

        #   # Plot the second histogram.
        #   axs[1].hist(temp, bins=bins2, edgecolor='black')
        #   axs[1].set_title(f'{i} a X_test_dist2mip')
        #   axs[1].set_xlabel('a Values')
        #   axs[1].set_ylabel('a Frequency')

        #   # Plot the third histogram.
        #   axs[2].hist(temp, bins=bins3*10, edgecolor='black')
        #   axs[2].set_title(f'{i} a X_test_dist2mip')
        #   axs[2].set_xlabel('a Values')
        #   axs[2].set_ylabel('a Frequency')

        #   # Display the plot.
        #   plt.tight_layout()
        #   plt.show()

        X_test_momentum = X_test["X_test_momentum"]
        X_test_refractive_index = X_test["X_test_refractive_index"]
        X_test_ckov = X_test["X_test_ckov"]
        X_test_mip_position = X_test["X_test_mip_position"]
        
        self.model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0002), loss="categorical_crossentropy", metrics=["accuracy"])

        # Print fields in the first vector of X_train
        print("Fields in the first vector of X_train:")
        print("X_train_map shape:", X_train_map.shape)
        print("X_train_momentum shape:", X_train_momentum.shape)
        print("X_train_refractive_index shape:", X_train_refractive_index.shape)
        print("X_train_ckov shape:", X_train_ckov.shape)
        print("X_train_mip_position shape:", X_train_mip_position.shape)

        # Print the first element of y_train
        print("First element of y_train:", y_train[0])

        # Train the model
        history = self.model.fit(
            x=[X_train_dist2mip, X_train_momentum, X_train_refractive_index, X_train_mip_position],
            y=y_train,
            validation_data=(
                [X_test_dist2mip,  X_test_momentum, X_test_refractive_index, X_test_mip_position],
                y_test
            ),
            batch_size=16,
            epochs=10,
            verbose=1
        )
        return history

    def evaluate_model(self, X_test, y_test):
        X_test_dist2mip = X_test["X_test_dist2mip"]
        X_test_momentum = X_test["X_test_momentum"]
        X_test_refractive_index = X_test["X_test_refractive_index"]
        X_test_ckov = X_test["X_test_ckov"]
        X_test_mip_position = X_test["X_test_mip_position"]
        loss, accuracy = self.model.evaluate(
            x=[X_test_dist2mip, X_test_momentum, X_test_refractive_index, X_test_mip_position],
            y=y_test,
            verbose=0
        )
        print(f"Test Loss: {loss:.4f}")
        print(f"Test Accuracy: {accuracy:.4f}")



    def plot_training_history(self, history):
        plt.figure(figsize=(12, 6))
        plt.subplot(1, 2, 1)
        plt.plot(history.history["loss"], label="Train Loss")
        plt.plot(history.history["val_loss"], label="Validation Loss")
        plt.xlabel("Epochs")
        plt.ylabel("Loss")
        plt.legend()

        plt.subplot(1, 2, 2)
        plt.plot(history.history["accuracy"], label="Train Accuracy")
        plt.plot(history.history["val_accuracy"], label="Validation Accuracy")
        plt.xlabel("Epochs")
        plt.ylabel("Accuracy")
        plt.legend()

        plt.show()

    def train(self, filename):
        self.load_data(filename)
        X_train, X_test, y_train, y_test = self.preprocess_data()
        X_train_dist2mip = np.array(X_train["X_train_dist2mip"])



        print(f" in  def train(self, filename) : X_train_dist2mip shape : {X_train_dist2mip.shape}")
        #X_train_dist2mip = X_train_dist2mip.reshape(X_train_dist2mip.shape[0], X_train_dist2mip.shape[1],, 1)
        #for dist in X_train_dist2mip:
        #  print(f"dist = {dist}")

        input_sequence_length = len(max(X_train_dist2mip, key=len))

        self.build_model(input_sequence_length = input_sequence_length)

        history = self.train_model(X_train, X_test, y_train, y_test)
        self.evaluate_model(X_test, y_test)
        self.plot_training_history(history)



    def plot_worst(model, y_test, X_test_map, X_test_momentum, X_test_refractive_index, X_test_ckov, X_test_mip_position, y_pred):
      # 1. Predict labels on validation data

      # 2. Calculate the difference between predicted and actual labels
      losses = tf.keras.losses.categorical_crossentropy(y_test, y_pred).numpy()

      # Sort the indices of the losses from highest to lowest
      sorted_indices = np.argsort(losses)[::-1]

      # Get the indices of the worst performing 10%
      worst_10_percent_indices = sorted_indices[:int(0.1*len(sorted_indices))]

      # Create figure and axes
      num_plots = len(worst_10_percent_indices)
      #fig, axes = plt.subplots(num_plots, 1, figsize=(8, 20))
      fig, axes = plt.subplots(num_plots,figsize=(8, 20))

      # Define mass categories
      mass_categories = ["pion", "kaon", "proton"]

      # 3. Create plots for these cases, including their feature information and predicted vs actual labels
      for i, index in enumerate(worst_10_percent_indices):
          # Get the map and corresponding information
          map_data = X_test_map[index, :, :]
          actual_mass_category = mass_categories[np.argmax(y_test[index])]

          print(f"y_test[index] = {y_test[index]}")

          predicted_mass_category = mass_categories[np.argmax(y_pred[index])]
          ckov = X_test_ckov[index]
          mip_position = X_test_mip_position[index]
          momentum = X_test_momentum[index]
          refractive_index = X_test_refractive_index[index]
          
          mass_actual = momentum * np.sqrt(refractive_index**2 * np.cos(ckov)*np.cos(ckov) - 1)
          
          # Check if the value is NaN (invalid Cherenkov angle)
          if np.isnan(mass_actual):
              mass_actual = "Invalid"

          # Plot the map
          axes[i].imshow(map_data, cmap='gray')

          # Add a red dot at the MIP position
          axes[i].plot(mip_position[0], mip_position[1], 'ro')

          # Set the title with the information
          axes[i].set_title(f"Actual Mass")#: {actual_mass_category}, Predicted Mass: {predicted_mass_category},\nMass: {mass_actual}, Mass_prob = {y_pred[index]} \nCKOV: {ckov}, MIP Position: {mip_position}, \nMomentum: {momentum}, Refractive Index: {refractive_index}")
          #
          axes[i].set_title(f"Actual Mass: {actual_mass_category}, Predicted Mass: {predicted_mass_category},\nMass: {mass_actual}, Mass_prob = {y_pred[index]} \nCKOV: {ckov}, MIP Position: {mip_position}, \nMomentum: {momentum}, Refractive Index: {refractive_index}")

          #axes[i].set_title(f"Actual Mass: {actual_mass_category}, Predicted Mass: {predicted_mass_category}, Mass: {mass_actual}\nCKOV: {ckov}, MIP Position: {mip_position}, Momentum: {momentum}, Refractive Index: {refractive_index}")
          axes[i].axis('off')

          print("\n")
          print(f"  Actual Mass: {actual_mass_category}, Predicted Mass: {predicted_mass_category},\n Mass: {mass_actual}, Mass_prob = {y_pred[index]} \n CKOV: {ckov}, MIP Position: {mip_position}, \n  Momentum: {momentum}, Refractive Index: {refractive_index}")
      # Adjust the spacing between subplots
      plt.tight_layout()

      # Show the plot
      plt.show()


# Usage example


#these three have .012 in stddev
# 3 percent occupancy:
classifier = MassClassifier(percentage_to_read = 25, resolution = 4) # pass percentage of dataset to read 
classifier.train("ParticleInfo.h5")


# only 1 percent occupancy :
#classifier2 = MassClassifier(percentage_to_read = 5, resolution = 4) # pass percentage of dataset to read 
#classifier2.train("ParticleInfo_1percent.h5")


#only .3 percent occupancy :
#classifier2 = MassClassifier(percentage_to_read = 20, resolution = 4) # pass percentage of dataset to read 
#classifier2.train("ParticleInfo_03percent.h5")



# Less stddev in ckov photons : .07 std

  #only .3 percent occupancy :
#classifier2 = MassClassifier(percentage_to_read = 20, resolution = 4) # pass percentage of dataset to read 
#classifier2.train("ParticleInfo_03percent_std07.h5")


Number of particles: 2450
plot_maps failed due to error : name 'y_test' is not defined
maps shape = (2450, 576, 640)
mip_position_array shape = (2450, 2)
distances_map_list shape = (2450,)
 X_dist2mip shape befoer pad = (2450,)
 X_dist2mip shape after pad = (2450, 36)


<ipython-input-13-f9bb0cd02ba0>:251: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  print(f" X_dist2mip shape befoer pad = {np.array(X_dist2mip).shape}")


 in  def train(self, filename) : X_train_dist2mip shape : (1960, 36)
plot_maps failed due to error : name 'y_test' is not defined
maps shape = (2450, 576, 640)
mip_position_array shape = (2450, 2)
distances_map_list shape = (2450,)
 X_dist2mip shape befoer pad = (2450,)
 X_dist2mip shape after pad = (2450, 36)


<ipython-input-13-f9bb0cd02ba0>:251: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  print(f" X_dist2mip shape befoer pad = {np.array(X_dist2mip).shape}")


In [ ]:
1X_train_dist2mip_padded = pad_sequences(X_train_dist2mip, maxlen=input_sequence_length, padding='post', value=0.0)


In [ ]:
# Plot training loss and validation loss
import matplotlib.pyplot as plt
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

# Plot training accuracy and validation accuracy
plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Make predictions on the validation data


y_train_pred = model.predict([X_train_map, X_train_momentum, X_train_refractive_index, X_train_mip_position])

# Convert the predictions from categorical back to original labels
y_train_pred_classes = np.argmax(y_train_pred, axis=1)

# Calculate the confusion matrix
#cm = confusion_matrix(y_train, y_train_pred_classes)

# Calculate the confusion matrix
rounded_labels=np.argmax(y_train, axis=1)

cm = confusion_matrix(rounded_labels, y_train_pred_classes)

# Use seaborn to visualize the confusion matrix
plt.figure(figsize=(10, 7))
sns.heatmap(cm, annot=True, fmt="d")
plt.title('Confusion matrix Training Data')
plt.ylabel('Actual label')
plt.xlabel('Predicted label')
plt.show()


In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Make predictions on the validation data
y_val_pred = model.predict([X_test_map, X_test_momentum, X_test_refractive_index, X_test_mip_position])

# Convert the predictions from categorical back to original labels
y_val_pred_classes = np.argmax(y_val_pred, axis=1)

# Calculate the confusion matrix
rounded_labels=np.argmax(y_test, axis=1)

cm = confusion_matrix(rounded_labels, y_val_pred_classes)

# Use seaborn to visualize the confusion matrix
plt.figure(figsize=(10, 7))
sns.heatmap(cm, annot=True, fmt="d")
plt.title('Confusion matrix Validation Data')
plt.ylabel('Actual label')
plt.xlabel('Predicted label')
plt.show()
